In [ ]:
import os
import re
import importlib

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from skimage.measure import block_reduce
from skimage.transform import resize

import data_utils

## Load and preprocess the data

In [ ]:
importlib.reload(data_utils);

In [ ]:
path = "/Users/jkamalu/Downloads/BosphorusDB"

# Load the data and store in dictionary
lookup_table = data_utils.load_data(path)

# Extract lists from dictionary and create inverse index
lookup_table, landmarks, face_data = data_utils.convert_data(lookup_table)

# Pad the face data to max width and height
data_utils.padding_data(face_data)

# Convert to numpy arrays
landmarks, face_data = map(np.array, (landmarks, face_data))

In [ ]:
# Adjust and scale all channels

N, W, H, channels = face_data.shape

for i in range(face_data.shape[0]):
    for channel in range(channels):
        lowest, low = np.sort(np.unique(face_data[i, :, :, channel]))[:2]
        face_data[i, :, :, channel][face_data[i, :, :, channel] == lowest] = low

scaler = MinMaxScaler()
scaler.fit(face_data[:, :, :, :].reshape(-1, channels))
scaled_z = scaler.transform(face_data[:, :, :, :].reshape(-1, channels)).reshape((N, W, H, channels))
face_data[:, :, :, :] = scaled_z

In [ ]:
# Visualize the face data before resizing and downsampling
data_utils.visualize_z(lookup_table, face_data)

In [ ]:
# Resize and downsample the images to 80 x 60 from 267 x 194

face_data = resize(face_data, (face_data.shape[0], 264, 192, face_data.shape[3]), preserve_range=True)
face_data = block_reduce(face_data, block_size=(1, 3, 3, 1), func=np.max)
print("face_data.shape:", face_data.shape)

In [ ]:
# Visualize the face data after resizing and downsampling
data_utils.visualize_z(lookup_table, face_data)

## Build the model

In [ ]:
from keras import Model, Sequential
from keras.layers import Input, Dense, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose, UpSampling2D
from keras.layers import Flatten, Reshape
from keras import backend as K

In [ ]:
class Params:

    drop_rate = 0.2
    n_filters = 32
    n_rand = 100
    
    def __init__(self, X, X_cond=None):
        _, self.W, self.H, self.n_channels = X.shape
        _, self.n_cond = X_cond.shape if X_cond else (None, 0)

In [ ]:
class CGAN:
    
    def __init__(self, params):
        self.params = params
        self.build_D()
        self.build_G()
        self.build_GD()
        self.compile_all()
    
    def build_D(self):
        d_input = Input((self.params.W, params.H, params.n_channels))
        
        d_layer = Conv2D(self.params.n_filters, (4, 4), padding="same", activation="relu")(d_input)
        d_layer = MaxPooling2D()(d_layer)
        d_layer = Dropout(rate=self.params.drop_rate)(d_layer)
        
        d_layer = Conv2D(self.params.n_filters * 2, (4, 4), padding="same", activation="relu")(d_layer)
        d_layer = MaxPooling2D()(d_layer)
        d_layer = Dropout(rate=self.params.drop_rate)(d_layer)
        
        d_layer = Conv2D(self.params.n_filters * 4, (4, 4), padding="same", activation="relu")(d_layer)
        d_layer = MaxPooling2D()(d_layer)
        d_layer = Dropout(rate=self.params.drop_rate)(d_layer)        
        
        d_layer = Flatten()(d_layer)
        d_layer = Dense(1)(d_layer)
        
        self.D = Model(inputs=d_input, outputs=d_layer)
        
        return self.D
        
    def build_G(self):
        g_input = Input((self.params.n_rand + self.params.n_cond, ))
        
        g_layer = Dense((self.params.W // 8) * 
                        (self.params.H // 8) *
                        self.params.n_filters * 8, activation="relu")(g_input)
        g_layer = BatchNormalization()(g_layer)
        g_layer = Reshape((self.params.W // 8, 
                          self.params.H // 8,
                          self.params.n_filters * 8))(g_layer)
        g_layer = Dropout(rate=self.params.drop_rate)(g_layer)
        
        g_layer = UpSampling2D()(g_layer)
        g_layer = Conv2DTranspose(self.params.n_filters * 4, (4, 4), padding="same", activation="relu")(g_layer)
        g_layer = BatchNormalization()(g_layer)
        g_layer = Dropout(rate=self.params.drop_rate)(g_layer)
        
        g_layer = UpSampling2D()(g_layer)
        g_layer = Conv2DTranspose(self.params.n_filters * 2, (4, 4), padding="same", activation="relu")(g_layer)
        g_layer = BatchNormalization()(g_layer)
        g_layer = Dropout(rate=self.params.drop_rate)(g_layer)        
        
        g_layer = UpSampling2D()(g_layer)
        g_layer = Conv2DTranspose(self.params.n_filters, (4, 4), padding="same", activation="relu")(g_layer)
        g_layer = BatchNormalization()(g_layer)
        g_layer = Dropout(rate=self.params.drop_rate)(g_layer)        
        
        g_layer = Conv2DTranspose(self.params.n_channels, (4, 4), padding="same", activation="sigmoid")(g_layer)
        g_layer = BatchNormalization()(g_layer)
        g_layer = Dropout(rate=self.params.drop_rate)(g_layer)
        
        g_image = g_layer
        
        self.G = Model(inputs=g_input, outputs=g_image)
        
        return self.G
        
    def build_GD(self):
        self.GD = Sequential()
        
        self.GD.add(self.G)
        
        self.D.trainable = False
        self.GD.add(self.D)
        
        return self.GD
    
    def compile_all(self):
        self.D.compile(optimizer="sgd", loss="binary_crossentropy", metrics=["accuracy"])
        
        self.GD.compile(optimizer="sgd", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
params = Params(face_data[:, :, :, :3])
cgan = CGAN(params)